# Implementing into pipeline

Now we take the code I've made to filter and generate traces and implement it into the framework of Alex's pipeline. 

First, the filter_and_downsample.py file used to read in AOSLO data, which takes the .mat files direclty, downsamples them to 1000 frames/second (1 ms), filters them (sliding window) and saves them as a .mat files in the format: (number of paths, number of timesteps, 2 = number dimensions of path)

In [ ]:
# Python script to remove erroneous datapoints and downsample high frequency AOSLO data
#  takes all .mat files in <data_dir> filters them, and downsamples them.
#
# Saves a file in output_dir with the paths in a .mat file in the format:
#    (number of paths, number of timesteps, 2 = number dimensions of path)

import numpy as np
import os
import sys
from scipy.io import loadmat, savemat
from scipy.interpolate import interp1d
from pykalman import KalmanFilter


# Parameters
print 'Ensure that parameters are correct before running'

pix_to_deg = 429.
#pix_to_deg = 392. # See pix_to_deg.txt for correct value
data_desc = '8_19_2014_aoslo_data'

resample_dt = 0.001 # Get samples with this time between samples

show_plots = True # Make true to show plots to help debug
if show_plots:
    import matplotlib.pyplot as plt


# Moving average parameters
a = 7 # Number of timesteps to do moving average over

# Kalman smoothing parameters
sigma_p = 30. # Initial velocity standard deviation arcmin/s
sigma_t = 0.04 # Hidden state transition noise
sigma_o = 0.5 # Observation noise

# File I/O Params
data_dir = 'data_high_freq/'

fn_s = [fn for fn in os.listdir(data_dir) if fn.endswith('.mat')]

output_dir = 'resampled_data/'


def process_data(filename):
    """
    Preprocesses data
    filename - .mat filename to high frequency aoslo data
    Applies the following transformations:
    -- Moving average filter to damp out large jumps
    -- Applies kalman smoother to damp out tumbling E artifacts
    Returns filtered path in matrix form:
    (N_T, 2)
    """
    data = loadmat(os.path.join(data_dir, filename))

    scale = 1. / pix_to_deg * 60. # Conversion from pixels to arcmins

    t = data['timeaxis_secs'][:, 0]
    t = t - t[0]
    dt = t[1] - t[0]
    f0 = dt ** -1 # Data frequency
    
    print 'Time interval = %0.2f s' % t[-1]

    xy = data['frameshifts_strips_spline'] * scale
    x = xy[:, 0]
    y = xy[:, 1]
    x = x - x[0]
    y = y - y[0]

    # Prefilter data (i.e. large jumps)
    xf, yf = moving_average(x, y, a = a)
    xf = xf - xf[0]
    yf = yf - yf[0]
    xyf = np.zeros_like(xy)
    xyf[:, 0] = xf
    xyf[:, 1] = yf

    # Kalman Smoothing

    e1 = np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0], 
                   [0, 0, 1, 0],
                   [0, 0, 0, 0]])

    e2 = np.array([[0, 0, 0, 0], 
                   [0, 1, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

    kf = KalmanFilter(transition_matrices = [[1, dt, 0, 0], 
                                             [0, 1, 0, 0], 
                                             [0, 0, 1, dt],
                                             [0, 0, 0, 1]],
                      transition_covariance = sigma_t * e2,
                      observation_matrices = [[1, 0, 0, 0], 
                                              [0, 0, 1, 0]],
                      observation_covariance = sigma_o * np.eye(2),
                      initial_state_mean = np.zeros(4),
                      initial_state_covariance = sigma_p * e2
                      )
                    

    (smoothed_state_means, smoothed_state_covariances) = kf.smooth(xyf)

    XYk = smoothed_state_means[:, (0,2)]
    Vk = smoothed_state_means[:, (1,3)]

    # Interpolation and downsampling
    fx = interp1d(t, XYk[:, 0])
    fy = interp1d(t, XYk[:, 1])

    # New times
    tp = np.arange(0, t[-1], resample_dt)

    xr = fx(tp)
    yr = fy(tp)

    xyr = np.zeros((tp.shape[0], 2))
    xyr[:, 0] = xr
    xyr[:, 1] = yr

    return xy, xyf, xyr
    
def debug_plots(xy, xyf, xyr):
    """
    Create plots of the data during the preprocessing
    xy - Raw data
    xyf - Raw data with a moving average filter
    xyr - resampled Kalman smoothed data
    """
    plt.figure(figsize = (15, 5))
    plt.subplot(1, 3, 1)
    plt.plot(xy[:, 0], xy[:, 1])
    plt.title('Raw Path')
    plt.xlabel('x (arcmin)')
    plt.ylabel('y (arcmin)')

    plt.subplot(1, 3, 2)
    plt.plot(xyf[:, 0], xyf[:, 1])
    plt.title('Moving Average Filtered Path')
    
    plt.subplot(1, 3, 3)
    plt.plot(xyr[:, 0], xyr[:, 1])
    plt.title('Kalman Smoothed Path')

    plt.show()


fn_s = fn_s[0:2]

for i, fn in enumerate(fn_s):
    xy, xyf, xyr = process_data(fn)
    try:
        resampled_paths
    except NameError:
        N_T_resample = xyr.shape[0]
        resampled_paths = np.zeros((len(fn_s), 
                                    N_T_resample, 2)).astype('float32')        

    resampled_paths[i, :, :] = xyr
    if show_plots:
        debug_plots(xyr, xyf, xy)
        

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

out_fn = 'resampled_paths'

d = {}
d['paths'] = resampled_paths
d['data_desc'] = data_desc
d['DT'] = resample_dt

savemat(os.path.join(output_dir, out_fn), d)


Second, the path_generator.py file used to create different types of traces (diffusion or velocity model) and save them as .mat files.  
See original at https://github.com/alexanderganderson/fixated_eye_model/blob/master/utils/path_generator.py

In [ ]:
import numpy as np
from scipy.io import loadmat
import abc


class Center:

    def __init__(self, Lx, DC, DT):
        """
        Class that Implements a diffusing center in a box of size Lx
        Lx = linear dimension of square to diffuse in
        DC = diffusion constant
        DT = timestep size
        x = coordinates of the current location of the random walk,
            initialized as [0, 0]
        Initializes Center Object
        """
        self.Lx = Lx
        self.DC = DC
        self.m0 = np.array([0, 0], dtype='float64')  # current position
        self.DT = DT
        # The diffusion is biased towards the center by taking a product of
        # gaussians
        # A product of gaussians is also a gaussian with mean, sdev given as
        # (mn, sn)
        self.m1 = np.array([0, 0], dtype='float64')  # center of image
        # Standard deviation for diffusion
        self.s0 = np.sqrt(self.DC * self.DT)
        self.s1 = self.Lx / 4  # Standard Deviation for centering gaussian
        self.sn = 1 / np.sqrt(1 / self.s0 ** 2 + 1 / self.s1 ** 2)

    def advance(self):
        """
        Updates location according to a random walk that stays within a box
        """

        self.mn = (
            self.m0 / self.s0 ** 2 + self.m1 / self.s1 ** 2) * self.sn ** 2
        while(True):
            # Note that for 2d diffusion, each component's variance is half the
            #   variance of the overall step length
            temp = self.mn + \
                np.random.normal(size=2, scale=self.sn / np.sqrt(2))
            if (temp[0] > - self.Lx / 2
                    and temp[0] < self.Lx / 2
                    and temp[1] > - self.Lx / 2
                    and temp[1] < self.Lx / 2):
                self.m0 = temp
                break

    def get_center(self):
        return self.m0

    def reset(self):
        self.m0 = np.array([0, 0], dtype='float64')

    def __str__(self):
        return str(self.x)


class PathGenerator():
    __metaclass__ = abc.ABCMeta

    @abc.abstractmethod
    def __init__(self, N_T, *args):
        """
        Initialize a 2d Path Generator
        """
        self.N_T = N_T

    @abc.abstractmethod
    def gen_path(self):
        """
        Generates a 2d path with N_T timesteps starting at (0,0)
        """
        return np.zeros((2, self.N_T))

    @abc.abstractmethod
    def mode(self):
        """
        Returns a string describing the path
        """
        return ' '


class DiffusionPathGenerator(PathGenerator):

    def __init__(self, N_T, Lx, DC, DT):
        """
        Creates a path generator that does bounded diffusion
        N_T - number of timesteps to generate path
        Lx - window that the diffusion is restricted to
        DC - diffusion constant
        DT - timestep
        """
        PathGenerator.__init__(self, N_T)
        self.c = Center(Lx, DC, DT)

    def gen_path(self):
        self.c.reset()
        path = np.zeros((2, self.N_T))
        for i in range(self.N_T):
            path[:, i] = self.c.get_center()
            self.c.advance()
        return path

    def mode(self):
        return 'Diffusion'


class ExperimentalPathGenerator(PathGenerator):

    def __init__(self, N_T, filename, DT):
        """
        Creates a path generator that uses real experimental data
        filename - filename for pkl file that contains an array of paths
              data['paths'] = (N_runs, number of timesteps, 2)
              2 - number of dimensions of path eg. x,y
        """
        PathGenerator.__init__(self, N_T)
        self.data = loadmat(filename)
        self.DT = self.data['DT'][0, 0]
        self.paths = self.data['paths']
        self.N_runs, self.N_T_data, _ = self.paths.shape

        if not self.DT == DT:
            raise ValueError('Data timestep doesnt match simulation timestep')
        if self.N_T > self.N_T_data:
            pass  # raise ValueError('Simulation has more timesteps than data')

    def gen_path(self):
        """
        Generate a path from the data
        """
        q = np.random.randint(self.N_runs)
        st = np.random.randint(self.N_T_data - self.N_T)
        pat = self.paths[q, st:(st + self.N_T), :].transpose()
        pat[0] = pat[0] - pat[0][0]
        pat[1] = pat[1] - pat[1][0]
        return pat

    def mode(self):
        return 'Experimental_data'


if __name__ == '__main__':
    fn = 'data/resampled_paths.mat'
    pg = ExperimentalPathGenerator(100, fn, 0.001)
    path = pg.gen_path()
    print path
Status API Training Shop Blog About Pricing
© 2015 GitHub, Inc. Terms Privacy Security Contact Help